In [29]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Rescaling
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, LayerNormalization
import tensorflow_addons as tfa
import pickle
from tensorflow.keras.applications import Xception
from tensorflow.keras.regularizers import L2
from keras.models import Model

In [30]:
train_path = "./train"
test_path = "./test"


In [31]:
def process_data(img_dims, batch_size):
  train_datagen = ImageDataGenerator(rescale=1./255)
  test_datagen = ImageDataGenerator(rescale=1./255)

  train_gen = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(img_dims, img_dims),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
  )

  test_gen = test_datagen.flow_from_directory(
      directory=test_path,
      target_size=(img_dims, img_dims),
      batch_size=batch_size,
      class_mode='binary',
      shuffle=True
  )

  return (train_gen, test_gen)

In [32]:
img_dims = 160
epochs=10
batch_size=16

train_gen, test_gen = process_data(img_dims, batch_size)

Found 12134 images belonging to 2 classes.
Found 2615 images belonging to 2 classes.


In [33]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [34]:
num_classes = 2
input_shape = (160, 160, 3)

In [35]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 16
num_epochs = 1
image_size = 160  # We'll resize input images to this size
patch_size = 80  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [36]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [37]:
class Patches(layers.Layer):
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'patch_size': self.patch_size,
        })
        return config

    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, batch_size, 80, 80, 3])
        print("patches.shape : ", patches.shape)
        return patches

In [54]:
class PatchEncoder(layers.Layer):
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'num_patches': self.num_patches,
            'projection': self.projection,
            'position_embedding': self.position_embedding,
        })
        return config

    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches

        # regularizer = L2(l2=0.001)

        # xception = Xception(input_shape=(80,80,3),weights='imagenet', include_top=False)

        # for layer in xception.layers:
        #     layer.trainable = False

        # x = Flatten()(xception.output)
        # prediction = Dense(1024, activation='relu', kernel_regularizer=regularizer)(x)
        # prediction = Dense(512, activation='relu', kernel_regularizer=regularizer)(prediction)
        # prediction = Dense(256, activation='relu', kernel_regularizer=regularizer)(prediction)
        # prediction = Dense(units=projection_dim, activation='relu', kernel_regularizer=regularizer)(prediction)

        # model = Model(inputs=xception.input, outputs=prediction)

        # self.projection = CustomXception
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        print("Patch.shape : ", patch.shape)
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = CustomXception()(patch) + self.position_embedding(positions)
        return encoded

In [64]:
class CustomXception(layers.Layer):
  def __init__(self):
    super(CustomXception, self).__init__()

  def concatOutput(self, ):


  def call(self, patches):
    print("inside custom xception layer : ", patches.shape)

    regularizer = L2(l2=0.001)

    xception = Xception(input_shape=(80, 80, 3), weights='imagenet', include_top=False)

    for layer in xception.layers:
      layer.trainable = False
    
    x = Flatten()(xception.output)
    prediction = Dense(1024, activation='relu', kernel_regularizer=regularizer)(x)
    prediction = Dense(512, activation='relu', kernel_regularizer=regularizer)(prediction)
    prediction = Dense(256, activation='relu', kernel_regularizer=regularizer)(prediction)
    prediction = Dense(units=64, activation='relu', kernel_regularizer=regularizer)(prediction)

    model = Model(inputs=xception.input, outputs=prediction)

    # outputs = tf.placeholder(tf.float32)
    finalOutput = None

    for id, patch in enumerate(patches):
      print('inside custom layer for loop : ', patch.shape)
      output = model(patch)
      if (id>0):
        finalOutput = tf.concat([finalOutput, output], axis=0)
      if (id == 0):
        finalOutput=output
      # outputs = tf.concat([outputs, output], axis=0)
    return finalOutput
      

IndentationError: expected an indented block (3286687708.py, line 8)

In [65]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = Patches(patch_size)(inputs)

    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [66]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            'accuracy',
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        train_gen,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=test_gen,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy = model.evaluate(test_gen)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history


vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

patches.shape :  (None, None, 80, 80, 3)
Patch.shape :  (None, None, 80, 80, 3)
inside custom xception layer :  (None, None, 80, 80, 3)
inside custom layer for loop :  (None, 80, 80, 3)


InaccessibleTensorError: in user code:

    /tmp/ipykernel_9105/3591669408.py:38 call  *
        encoded = CustomXception()(patch) + self.position_embedding(positions)
    /tmp/ipykernel_9105/4087729865.py:29 call  *
        outputs = tf.convert_to_tensor(outputs)
    /home/dhiraj/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    /home/dhiraj/.local/lib/python3.8/site-packages/tensorflow/python/framework/ops.py:1430 convert_to_tensor_v2_with_dispatch
        return convert_to_tensor_v2(
    /home/dhiraj/.local/lib/python3.8/site-packages/tensorflow/python/framework/ops.py:1436 convert_to_tensor_v2
        return convert_to_tensor(
    /home/dhiraj/.local/lib/python3.8/site-packages/tensorflow/python/profiler/trace.py:163 wrapped
        return func(*args, **kwargs)
    /home/dhiraj/.local/lib/python3.8/site-packages/tensorflow/python/framework/ops.py:1566 convert_to_tensor
        ret = conversion_func(value